1⃣️【第十四周作业】

1. 通过langchain实现特定主题聊天系统，支持多轮对话。
2. 借助langchain实现图书管理系统开发扩展，通过图书简介为借阅读者提供咨询。

In [1]:
from dotenv import find_dotenv, load_dotenv
import os
from langchain_openai import ChatOpenAI # LLM调用封装
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage # 对话角色：user、assistant、system
from langchain_core.output_parsers import StrOutputParser

# 1. 导入必要包
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 导入template中传递聊天历史信息的“占位”类
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
# 1. 通过langchain实现特定主题聊天系统，支持多轮对话。
# 2. 借助langchain实现图书管理系统开发扩展，通过图书简介为借阅读者提供咨询。
if __name__ == '__main__':
    load_dotenv(find_dotenv())

    # Initialize the OpenAI Chat model
    model = ChatOpenAI(
        model="qwen-plus",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key="sk-**************************",
        temperature=0.7
    )
 
    # 带有占位符的prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一个专注于{theme}的专家助手，只能回答相关领域问题,尽你所能使用{lang}回答所有问题。"),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    parser = StrOutputParser()
    # chain 构建
    chain =  prompt | model | parser
    
    # 定制存储消息的dict
    # key:  sessionId会话Id（资源编号）区分不同用户或不同聊天内容
    # value: InMemoryChatMessageHistory存储聊天信息list对象
    store = {}

    # 定义函数：根据sessionId获取聊天历史（callback 回调）
    # callback 系统调用时被执行代码
    def get_session_hist(session_id):
        # 以sessionid为key从store中提取关联聊天历史对象
        if session_id not in store:
            store[session_id] = InMemoryChatMessageHistory() # 创建
        return store[session_id] # 检索 （langchain负责维护内部聊天历史信息）

    # 在chain中注入聊天历史消息
    # 调用chain之前，还需要根据sessionID提取不同的聊天历史
    with_msg_hist = RunnableWithMessageHistory(
        chain, 
        get_session_history=get_session_hist,
        input_messages_key="messages")  # input_messages_key 指明用户消息使用key
    
    # session_id
    session_id = "abc123"

    flag = 1
    while flag:
        # 用户输入
        user_input = input('用户输入的Message：')
        print('User question:',user_input)
        if "结束会话" == user_input :
            flag = 0
            continue
        # 调用注入聊天历史的对象
        response = with_msg_hist.invoke(
            {
                "messages":[HumanMessage(content=user_input)],
                "lang":"中文",
                "theme":"电风扇"
            },
            config={'configurable':{'session_id': session_id}})
        print('AI Message:', response)

User question: 电风扇的种类
AI Message: 电风扇的种类繁多，根据不同的分类标准可以分为以下几类：

---

### 一、按用途和安装方式分类：
1. **落地扇**  
   - 放置在地面使用，通常有可调节高度的支架和底座，适合大范围送风。
2. **台扇（桌面扇）**  
   - 小型风扇，适合放在桌子、床头柜等地方使用，适用于局部降温。
3. **壁扇**  
   - 安装在墙壁上，节省空间，常用于工厂、仓库或厨房。
4. **吊扇**  
   - 悬挂在天花板上，广泛用于家庭、餐厅、商场等空间较大的场所。
5. **工业风扇 / 工业大风扇**  
   - 大型风扇，用于厂房、车间、仓库等大型空间通风降温。
6. **空调扇（冷风扇）**  
   - 具备加湿、制冷功能，通过水蒸发实现降温，但不是真正的空调。

---

### 二、按送风方式分类：
1. **轴流风扇**  
   - 风沿轴向流动，结构简单，效率高，常见于落地扇、壁扇等。
2. **离心风扇**  
   - 风从侧面进入，从另一侧排出，常用于空调扇或需要静音的场合。
3. **贯流风扇（横流风扇）**  
   - 风横向穿过叶片，送风范围广，常用于空调扇、暖风机等。

---

### 三、按控制方式分类：
1. **机械式风扇**  
   - 使用旋钮或开关手动调节风速。
2. **电子控制风扇**  
   - 可调速、定时、遥控等功能齐全，智能化程度高。
3. **直流变频风扇**  
   - 节能环保，噪音低，可通过变频技术调节风速。

---

### 四、按外观与结构分类：
1. **传统扇叶风扇**  
   - 带有金属或塑料扇叶的传统风扇。
2. **无叶风扇（空气增压扇）**  
   - 没有外露扇叶，安全性高，清洁方便，科技感强。
3. **塔式风扇**  
   - 立柱状设计，节省空间，美观现代，多用于室内环境。

---

### 五、按使用场景分类：
1. **家用风扇**  
   - 如台扇、落地扇、空调扇等，适合家庭使用。
2. **商用风扇**  
   - 如吊扇、壁扇、工业风扇，适用于商场、办公室等。
3. **工业风扇**  
   - 专为工厂、车间设计，强调耐用性和大风量。

---

如果你